# Ant-Fungi Hybrid Optimization Algorithm

Ant Colony Optimization (ACO) algorithm to incorporate some of the efficient path-finding strategies used by fungi (specifically slime mold or the mycelium network). This hybrid approach could leverage the speed of ants and the adaptive resilience of fungi, resulting in a potentially more powerful optimization algorithm.

Concept: Ant-Fungi Hybrid Optimization Algorithm (AFHO)
To design an algorithm that blends the strengths of both ACO and fungi-inspired methods, let's outline how this can be achieved:

## Ant-like Exploration and Pheromone Trails:

We'll maintain the classic ACO behavior where artificial ants traverse a network and deposit pheromones to communicate with each other, reinforcing the shortest paths.
Ants will prioritize paths with higher pheromone levels but also explore new paths with a certain probability, balancing exploitation and exploration.
Fungi-like Network Adaptation:

Introduce a mycelium-inspired adaptive network structure where nodes (representing possible paths) can grow, shrink, or retract based on their utility.
Use a feedback mechanism similar to how fungi reinforce nutrient-rich paths and retract from inefficient ones.
Incorporate dynamic path adjustment that allows for the network to change in real-time, adapting to new conditions or obstacles.
Hybrid Pheromone and Nutrient Flow:

Instead of just relying on pheromones, introduce a "nutrient flow" concept similar to how fungi transport nutrients through their network.
Paths that are frequently used by ants receive both pheromone reinforcement and nutrient flow, allowing for adaptive reinforcement.

Key Benefits of the Hybrid Approach
Speed and Adaptability: Combines the rapid path-finding capabilities of ants with the adaptive network resilience of fungi.
Dynamic Optimization: Adapts to changing environments and finds optimal solutions even if the landscape changes.
Scalability: Can be applied to large, complex networks like logistics, supply chain management, or transportation systems.
This approach leverages the best of both worlds to provide a robust solution for complex optimization problems.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sko
from sko.ACA import ACA_TSP


Dataset is divided into 7 tables, one table for all orders that needs to be assigned a route – OrderList table, and 6 additional files specifying the problem and restrictions. For instance, the FreightRates table describes all available couriers, the weight gaps for each individual lane and rates associated. The PlantPorts table describes the allowed links between the warehouses and shipping ports in real world. Furthermore, the ProductsPerPlant table lists all supported warehouse-product combinations. The VmiCustomers lists all special cases, where warehouse is only allowed to support specific customer, while any other non-listed warehouse can supply any customer. Moreover, the WhCapacities lists warehouse capacities measured in number of orders per day and the WhCosts specifies the cost associated in storing the products in given warehouse measured in dollars per unit.



Order ID is ID of the order made by the customer, product ID is the specific product ID customer ordered.

In [4]:
import pandas as pd
import numpy as np
import random

# Precomputed driving distances from Google Maps
precomputed_distances = {
    ('Kigali', 'Butare'): 125,
    ('Bujumbura', 'Gitega'): 98.9,
    ('Juba', 'Wau'): 647,
    ('Nairobi', 'Arusha'): 274,
    ('Kampala', 'Bujumbura'): 720,
    ('Kampala', 'Juba'): 653,
    ('Kampala', 'Kigali'): 511,
    ('Kigali', 'Bujumbura'): 273,
    ('Nairobi', 'Bujumbura'): 1281,
    ('Dar es Salaam', 'Kigali'): 1444,
    ('Bujumbura', 'Dar es Salaam'): 1496,
    ('Bujumbura', 'Kisumu'): 1064,
    ('Nairobi', 'Tanzania'): 902,  
    ('Nairobi', 'Dar es Salaam'): 1012,  
    ('Nairobi', 'Kampala'): 663,  
    ('Nairobi', 'Wau'): 1868,  
    ('Nairobi', 'Kigali'): 1354,  
}

# Stopover cities
stopover_cities = {
    ('Nairobi', 'Tanzania'): 'Arusha',
    ('Nairobi', 'Dar es Salaam'): 'Mombasa',
    ('Nairobi', 'Kampala'): 'Kisumu',
    ('Nairobi', 'Wau'): 'Juba',
    ('Nairobi', 'Kigali'): 'Arusha',
}

# Expand city pairs to generate 1000 entries
random.seed(42)
expanded_city_pairs = random.choices(list(precomputed_distances.keys()), k=1000)

# Function to safely look up stopover cities
def get_stopover(origin, destination):
    return stopover_cities.get((origin, destination)) or stopover_cities.get((destination, origin)) or None

# Generate vehicle plates and stopover info
stopovers = []
vehicle_plates = []

for i, (origin, destination) in enumerate(expanded_city_pairs):
    stopovers.append(get_stopover(origin, destination))
    vehicle_plates.append(f"{origin[:3].upper()}-{destination[:3].upper()}-{str(i).zfill(4)}")

# Create DataFrame for city pairs
sub_saharan_cities = pd.DataFrame({
    'City of Origin': [pair[0] for pair in expanded_city_pairs],
    'Destination City': [pair[1] for pair in expanded_city_pairs],
    'Stopover City': stopovers,
    'Vehicle Plate': vehicle_plates,
    'Distance (km)': [precomputed_distances[pair] for pair in expanded_city_pairs]  # Use precomputed distance
})

# Vehicle types
np.random.seed(42)
vehicle_types = ['SUV', 'Truck', 'Saloon', 'Coupe', 'Hybrid']
vehicle_type = np.random.choice(vehicle_types, size=len(vehicle_plates))

# Speed limits per vehicle type
speed_ranges = {
    'SUV': (60, 120),
    'Truck': (40, 90),
    'Saloon': (50, 110),
    'Coupe': (70, 130),
    'Hybrid': (40, 100),
}

# Fuel consumption per 100 km (L/100km)
fuel_consumption_ranges = {
    'SUV': (10, 20),
    'Truck': (20, 35),
    'Saloon': (7, 15),
    'Coupe': (8, 18),
    'Hybrid': (5, 12),
}

# Generate speed and fuel consumption values
speeds = []
fuel_consumptions = []

for vt in vehicle_type:
    speed_min, speed_max = speed_ranges[vt]
    fuel_min, fuel_max = fuel_consumption_ranges[vt]
    
    speed = np.random.uniform(speed_min, speed_max)  # Random speed within range
    fuel_efficiency = np.random.uniform(fuel_min, fuel_max)  # Fuel usage per 100km
    
    speeds.append(speed)
    fuel_consumptions.append(fuel_efficiency)

# Create vehicle DataFrame
realistic_vehicles = pd.DataFrame({
    'Vehicle Plate': vehicle_plates,
    'Vehicle Type': vehicle_type,
    'Speed (km/h)': speeds,
    'Fuel Consumption (L/100km)': fuel_consumptions,
})

# Add traffic congestion levels
traffic_congestion = np.random.choice(['Low', 'Medium', 'High'], size=len(vehicle_plates))
realistic_vehicles['Traffic Congestion'] = traffic_congestion

# Adjust speed based on congestion (+/- 20 km/h)
speed_adjustments = {'Low': +20, 'Medium': 0, 'High': -20}
realistic_vehicles['Adjusted Speed (km/h)'] = realistic_vehicles.apply(
    lambda row: max(30, row['Speed (km/h)'] + speed_adjustments[row['Traffic Congestion']]), axis=1
)

# Compute travel time in hours (distance / adjusted speed)
merged_data = sub_saharan_cities.merge(realistic_vehicles, on='Vehicle Plate')

merged_data['Travel Time (hours)'] = merged_data['Distance (km)'] / merged_data['Adjusted Speed (km/h)']

# Compute fuel consumption for full trip
merged_data['Total Fuel Consumption (L)'] = (merged_data['Distance (km)'] * merged_data['Fuel Consumption (L/100km)']) / 100

# Define peak and off-peak times based on speed variations
merged_data['Peak Time'] = merged_data['Travel Time (hours)'] * np.random.uniform(1.1, 1.3, len(merged_data))
merged_data['Off-Peak Time'] = merged_data['Travel Time (hours)'] * np.random.uniform(0.7, 0.9, len(merged_data))

# Compute Travel Time Index (TTI)
# TTI = (Travel time under congestion) / (Free-flow travel time)
merged_data['Free-Flow Speed (km/h)'] = merged_data['Vehicle Type'].map(lambda vt: speed_ranges[vt][1])
merged_data['Free-Flow Time (hours)'] = merged_data['Distance (km)'] / merged_data['Free-Flow Speed (km/h)']
merged_data['Travel Time Index (TTI)'] = merged_data['Travel Time (hours)'] / merged_data['Free-Flow Time (hours)']

# Save the merged dataset
merged_data.to_csv('optimized_transport_data.csv', index=False)

# Display the first few rows
print("\nOptimized Transport Dataset (First 5 Rows):")
print(merged_data.head())



Optimized Transport Dataset (First 5 Rows):
  City of Origin Destination City Stopover City Vehicle Plate  Distance (km)  \
0      Bujumbura    Dar es Salaam          None  BUJ-DAR-0000         1496.0   
1         Kigali           Butare          None  KIG-BUT-0001          125.0   
2        Kampala        Bujumbura          None  KAM-BUJ-0002          720.0   
3        Nairobi           Arusha          None  NAI-ARU-0003          274.0   
4        Nairobi         Tanzania        Arusha  NAI-TAN-0004          902.0   

  Vehicle Type  Speed (km/h)  Fuel Consumption (L/100km) Traffic Congestion  \
0        Coupe    125.927097                   13.651332               High   
1       Hybrid     81.799049                   11.457496             Medium   
2       Saloon     92.434318                    8.220312             Medium   
3       Hybrid     74.577302                    9.247005             Medium   
4       Hybrid     65.447840                   10.155110             Medium   
